## Main Goal
In this project, we aim to examine the performance of various machine learning models for predicting sales of video games based on their attributes!  

### Data Understanding
This dataset contains a list of video games with sales greater than 100,000 copies. It was generated by a scrape of [vgchartz.com].
1. **Rank** - Ranking of overall sales
2. **Name** - The games name
3. **Platform** - Platform of the games release (i.e. PC,PS4, etc.)
4. **Year** - Year of the game's release
5. **Genre** - Genre of the game
6. **Publisher** - Publisher of the game
7. **NA_Sales** - Sales in North America (in millions)
8. **EU_Sales** - Sales in Europe (in millions)
9. **JP_Sales** - Sales in Japan (in millions)
10. **Other_Sales** - Sales in the rest of the world (in millions)
11. **Global_Sales** - Total worldwide sales.

In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv("datasets/vgsales.csv")

In [3]:
dataset.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
